In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_json('goemotions.json')
df.columns = ['post', 'emotion', 'sentiment']

In [3]:
df.head()

,post,emotion,sentiment
0,That game hurt.,sadness,negative
1,"You do right, if you don't care then fuck 'em!",neutral,neutral
2,Man I love reddit.,love,positive
3,"[NAME] was nowhere near them, he was by the Fa...",neutral,neutral
4,Right? Considering it’s such an important docu...,gratitude,positive


### 2.1. Process the dataset using feature_extraction.text.CountVectorizer to extract tokens/words and their frequencies.

In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df.post)

In [5]:
print(X.shape[1])

30449


### 2.2. Split the dataset into 80% for training and 20% for testing. 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, df[['emotion', 'sentiment']], test_size=0.2)

### 2.3. Train and test the following classifiers, for both the emotion and the sentiment classification, using word frequency as features.

In [7]:
performance_data = []
def train_model(model, model_name):
    for label in ['emotion', 'sentiment']:
        model.fit(X_train, y_train[label])
        y_pred = model.predict(X_test)
        performance_data.append(["%s."%model_name,"%s."%model, confusion_matrix(y_test[label], y_pred), classification_report(y_test[label], y_pred)])

In [8]:
def grid_search(model, model_name, params):
    clf = GridSearchCV(model, params, cv=2, n_jobs=-1)
    for label in ['emotion', 'sentiment']:
        clf.fit(X_train, y_train[label])
        param_str = ''
        for k, v in clf.best_params_.items():
            param_str += "{0}_{1}".format(k,v)
        y_pred = clf.best_estimator_.predict(X_test)
        performance_data.append(["{0}, {1}, {2}".format(model_name), confusion_matrix(y_test[label], y_pred), classification_report(y_test[label], y_pred)])

#### 2.3.1. Base-MNB: a Multinomial Naive Bayes Classifier (naive bayes.MultinomialNB.html) with the default parameters.

In [9]:
base_mnb = MultinomialNB()
train_model(base_mnb, 'MultinomialNB')

#### 2.3.2. Base-DT: a Decision Tree (tree.DecisionTreeClassifier) with the default parameters

In [10]:
base_dt = DecisionTreeClassifier()
train_model(base_dt, 'DecisionTreeClassifier')

#### 2.3.3. Base-MLP: a Multi-Layered Perceptron (neural network.MLPClassifier) with the default parameters.

In [ ]:
base_mlp = MLPClassifier()
train_model(base_mlp, 'MLPClassifier')

#### 2.3.4. Top-MNB: a better performing Multinomial Naive Bayes Classifier found using GridSearchCV

In [ ]:
mnb_params = {'alpha': [0.5, 0, 2]}
grid_search(base_mnb, 'MultinomialNB', mnb_params)

#### 2.3.5. Top-DT: a better performing Decision Tree found using GridSearchCV. 

In [ ]:
dt_params = {'criterion': ['gini', 'entropy'],
             'max_depth': [1, 3],
             'min_samples_split': [2, 3, 4]}
grid_search(base_dt, 'DecisionTreeClassifier', dt_params)

#### 2.3.6. Top-MLP: a better performing Multi-Layered Perceptron found using GridSearchCV.

In [ ]:
mlp_params = {'activation': ['identity', 'logistic', 'tanh', 'relu'],
              'hidden_layer_sizes': [(30, 50), (10, 10, 10)],
              'solver': ['adam', 'sgd']}
grid_search(base_mlp, 'MLPClassifier', mlp_params)

### 2.4. For each of the 6 classifiers above and each of the classification tasks (emotion or sentiment), produce and save the following information in a file called performance.

In [ ]:
with open('performance.txt', 'w') as f:
    for line in performance_data:
        line = list(map(str, line))
        f.write(",".join(line))

### 2.5. Use tf-idf instead of word frequencies and redo all substeps of 2.3 above – you can use TfidfTransformer for this. Display the results of this experiment.

In [ ]:
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, df[['emotion', 'sentiment']], test_size=0.2)

In [ ]:
performance_data = []
base_mnb = MultinomialNB()
train_model(base_mnb, 'MultinomialNB')

base_dt = DecisionTreeClassifier()
train_model(base_dt, 'DecisionTreeClassifier')

base_mlp = MLPClassifier()
train_model(base_mlp, 'MLPClassifier')

grid_search(base_mnb, 'MultinomialNB', mnb_params)

grid_search(base_dt, 'DecisionTreeClassifier', dt_params)

grid_search(base_mlp, 'MLPClassifier', mlp_params)

In [ ]:
pd.DataFrame(performance_data, columns=['Model', 'confusion matrix', 'classfication report'])

In [ ]:
from gensim.downloader import load
wordvec2model=load('word2vec-google-news-300')

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import pandas as pd
file = pd.read_json(open("goemotions.json", "r"))
words=file[0]
tokenizer=[word_tokenize(x) for x in words]

In [ ]:
list_tokens=[]
list_tokens=[k for k in tokenizer for i in k]
list_len=len(list_tokens)
print(f'There are {list_len} Reddit post tokens in goemotions.json')

In [ ]:
wordvec2model.vector_size

In [ ]:
vec_go=wordvec2model['go']
vec_go[:20]

In [ ]:
redditpositone=list_tokens[1]
print(redditpositone)

In [ ]:
vecter_list=[wordvec2model[word] for word in redditpositone if word in wordvec2model.index_to_key]

word_filtered_list=[word for word in redditpositone if word in wordvec2model.index_to_key]
word_vec= zip(word_filtered_list,vecter_list)
dict_tokens=dict(word_vec)
df=pd.DataFrame.from_dict(dict_tokens,orient='index')
df.head(len(dict_tokens))

In [ ]:
average=[]
k=0
for x in range(0,wordvec2model.vector_size-1):
    for j in range(0,len(dict_tokens)-1):
        k+=df[x][j]
    average.append(k/len(dict_tokens))


In [ ]:

df=pd.DataFrame(average,columns=['average'])

df.T.head()
